In [ ]:
#Imports
import json
import nltk
import numpy as np
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
import pandas as pd
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


In [ ]:
#Load models Sentiment
nltk.download('vader_lexicon')
nltk.download('punkt')
sentimentIntensityAnalyzer = SentimentIntensityAnalyzer()


pretrained_LM_path = "kornosk/bert-election2020-twitter-stance-biden-KE-MLM"

#Load models Stance
tokenizer = AutoTokenizer.from_pretrained(pretrained_LM_path)
stanceDetectionmodel = AutoModelForSequenceClassification.from_pretrained(pretrained_LM_path)

In [ ]:
#Settings
pd.set_option('display.width', 1000)

In [ ]:
#Load data
twitter_data = pd.read_csv('dataset/train.csv', encoding="unicode_escape", lineterminator='\r')

twitter_data.head(10)

In [ ]:
#Calculate Sentiment
def calculateSentiment(text):
    return sentimentIntensityAnalyzer.polarity_scores(text)['compound']

twitter_data['sentimentVader'] = twitter_data['Tweet'].apply(calculateSentiment)
display(twitter_data)

In [ ]:
# Calculate Stance Detection

id2label = {
    0: "AGAINST",
    1: "FAVOR",
    2: "NONE"
}

def calculateStance(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = stanceDetectionmodel(**inputs)
    print(inputs)
    prediction = torch.softmax(outputs[0], dim=1)[0].tolist()
    return id2label[np.argmax(prediction)]


twitter_data['stancePrediction'] = twitter_data['Tweet'].apply(calculateStance)

In [ ]:
display(twitter_data)